In [1]:
from __future__ import print_function
import os
from PIL import Image

import logging
import random
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
import numpy as np
import random
import torch
import torchvision
from torch.autograd import Variable
from torchvision import transforms, models
import torch.nn.functional as F
import torch.nn as nn
import torch
from resnet import resnet50_cbam

In [10]:
import torch.nn as nn
import torch


class Patch_CNN(nn.Module):
    def __init__(self, model, feature_size, classes_num):
        super(Patch_CNN, self).__init__()

        self.features = model
        self.max1 = nn.MaxPool2d(kernel_size=56, stride=56)
        self.max2 = nn.MaxPool2d(kernel_size=28, stride=28)
        self.max3 = nn.MaxPool2d(kernel_size=14, stride=14)
        self.num_ftrs = 2048 * 1 * 1
        self.elu = nn.ELU(inplace=True)

        self.classifier_concat = nn.Sequential(
            nn.BatchNorm1d(1024 * 3),
            nn.Linear(1024 * 3, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        self.conv_block1 = nn.Sequential(
            BasicConv(self.num_ftrs//4, feature_size, kernel_size=1, stride=1, padding=0, relu=True),
           # nn.Dropout(0.5,inplace=False),
            BasicConv(feature_size, self.num_ftrs//2, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier1 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )


    def forward(self, x):
        xf1, xf2, xf3, xf4, xf5 = self.features(x)
        print(xf3.size())
        xl1 = self.conv_block1(xf3)
        print(xl1.size())
        xl1 = self.max1(xl1)
        print(xl1.size())
        xl1 = xl1.view(xl1.size(0), -1)
        print(xl1.size())
        xc1 = self.classifier1(xl1)
        print(xc1.size())
      #  xc1 = torch.nn.Sigmoid(xc1)
        return xc1
    
    
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size,
                              stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes, eps=1e-5,
                                 momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x


In [3]:
import torch.nn as nn
import torch
class CNN(nn.Module):
    def __init__(self, feature_size, classes_num):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, padding=2),
            nn.BatchNorm2d(32, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, 3, 1, padding=1),
            nn.BatchNorm2d(64, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, 3, 1, padding=1),
            nn.BatchNorm2d(128, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, 3, 1, padding=1),
            nn.BatchNorm2d(256, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, 3, 1, padding=1),
            nn.BatchNorm2d(512, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.5, inplace=False),
            
            nn.Conv2d(512, 512, 2, 1),
            nn.BatchNorm2d(512, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5, inplace=False),

        )

        self.max1 = nn.MaxPool2d(kernel_size=13, stride=13)
        self.max2 = nn.MaxPool2d(kernel_size=28, stride=28)
        self.max3 = nn.MaxPool2d(kernel_size=14, stride=14)
        self.num_ftrs = 2048 * 1 * 1
        self.elu = nn.ELU(inplace=True)

        self.classifier_concat = nn.Sequential(
            nn.BatchNorm1d(1024 * 3),
            nn.Linear(1024 * 3, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )

        self.conv_block1 = nn.Sequential(
            BasicConv(self.num_ftrs//4, feature_size, kernel_size=1, stride=1, padding=0, relu=True),
            
            BasicConv(feature_size, self.num_ftrs//2, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier1 = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs//2),
            nn.Linear(self.num_ftrs//2, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Linear(feature_size, classes_num),
        )


    def forward(self, x):
        xf3 = self.features(x)
    

        xl1 = self.conv_block1(xf3)
 
        #xl1 = self.max1(xl1)
        xl1 = xl1.view(xl1.size(0), -1)
        xc1 = self.classifier1(xl1)
        xc1 = torch.Sigmoid(xc1)
        return xc1
    
    
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size,
                              stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes, eps=1e-5,
                                 momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x


In [4]:
import numpy as np
import random
import torch
import torchvision
from torch.autograd import Variable
from torchvision import transforms, models
import torch.nn.functional as F
from Resnet import *

def cosine_anneal_schedule(t, nb_epoch, lr):
    cos_inner = np.pi * (t % (nb_epoch))  # t - 1 is used when t has 1-based indexing.
    cos_inner /= (nb_epoch)
    cos_out = np.cos(cos_inner) + 1

    return float(lr / 2 * cos_out)


def load_model(model_name, pretrain=True, require_grad=True):
    print('==> Building model..')
    if model_name == 'resnet50_Patch_CNN':
        net = resnet50(pretrained=pretrain)
        for param in net.parameters():
            param.requires_grad = require_grad
        net = Patch_CNN(net, 512, 2)
    if model_name == 'CNN':
        net = CNN(512, 2)
    if model_name == 'resnet50_cbam':
        net = resnet50_cbam(pretrained=pretrain)
        for param in net.parameters():
            param.requires_grad = require_grad
        net = Patch_CNN(net, 512, 2)
    return net


def model_info(model): 
    n_p = sum(x.numel() for x in model.parameters())  
    n_g = sum(x.numel() for x in model.parameters() if x.requires_grad)  
    print('\n%5s %50s %9s %12s %20s %12s %12s' % ('layer', 'name', 'gradient', 'parameters', 'shape', 'mu', 'sigma'))
    for i, (name, p) in enumerate(model.named_parameters()):
        name = name.replace('module_list.', '')
        print('%5g %50s %9s %12g %20s %12.3g %12.3g' % (
            i, name, p.requires_grad, p.numel(), list(p.shape), p.mean(), p.std()))
    print('Model Summary: %g layers, %g parameters, %g gradients\n' % (i + 1, n_p, n_g))


def patch_transformation(images, n):
    l = []
    for a in range(n):
        for b in range(n):
            l.append([a, b])
    block_size = 448 // n
    rounds = n ** 2
    random.shuffle(l)
    patchrandom = images.clone()
    for i in range(rounds):
        x, y = l[i]
        temp = patchrandom[..., 0:block_size, 0:block_size].clone()
        patchrandom[..., 0:block_size, 0:block_size] = patchrandom[..., x * block_size:(x + 1) * block_size,
                                                y * block_size:(y + 1) * block_size].clone()
        patchrandom[..., x * block_size:(x + 1) * block_size, y * block_size:(y + 1) * block_size] = temp

    return patchrandom


def test(net, criterion, batch_size):
    net.eval()
    use_cuda = torch.cuda.is_available()
    test_loss = 0
    correct = 0
    correct_com = 0
    total = 0
    idx = 0
    device = torch.device("cuda:3")

    transform_test = transforms.Compose([
        transforms.Scale((500, 500)),
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    testset = torchvision.datasets.ImageFolder(root='/home/Data/FF/C40/DFt/val',
                                               transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

    for batch_idx, (inputs, targets) in enumerate(testloader):
        idx = batch_idx
        if use_cuda:
            inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        output_1= net(inputs)

        loss = criterion(output_1, targets)

        test_loss += loss.item()
        _, predicted = torch.max(output_1.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        if batch_idx % 50 == 0:
            print('Step: %d | Loss: %.3f | Acc: %.3f%% (%d/%d)' % (
            batch_idx, test_loss / (batch_idx + 1), 100. * float(correct) / total, correct, total, ))

    test_acc = 100. * float(correct) / total
    test_acc_en = 100. * float(correct_com) / total
    test_loss = test_loss / (idx + 1)

    return test_acc, test_acc_en, test_loss





In [11]:
def train(nb_epoch, batch_size, store_name, resume=False, start_epoch=0, model_path=None):
    # setup output
    exp_dir = store_name
    try:
        os.stat(exp_dir)
    except:
        os.makedirs(exp_dir)

    use_cuda = torch.cuda.is_available()
    print(use_cuda)

    # Data
    print('==> Preparing..')
    transform_train = transforms.Compose([
        transforms.Scale((500, 500)),
        transforms.RandomCrop(448, padding=8),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    trainset = torchvision.datasets.ImageFolder(root='/home/Data/FF/C40/DFt/train', transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    # Model
    if resume:
        net = torch.load(model_path)
    else:
        net = load_model(model_name='resnet50_Patch_CNN', pretrain=True, require_grad=True)
    netp = torch.nn.DataParallel(net, device_ids=[3])

    # GPU
    device = torch.device("cuda:3")
    net.to(device)
    # cudnn.benchmark = True

    CELoss = nn.CrossEntropyLoss()
    optimizer = optim.SGD([
        {'params': net.classifier_concat.parameters(), 'lr': 0.0005},
        {'params': net.conv_block1.parameters(), 'lr': 0.0005},
        {'params': net.classifier1.parameters(), 'lr': 0.0005},#0.002
        {'params': net.features.parameters(), 'lr': 0.00005}#0.0002

    ],
        momentum=0.9, weight_decay=5e-4)
    

    max_val_acc = 0
    lr = [0.0005, 0.0005, 0.0005, 0.00005]

    for epoch in range(start_epoch, nb_epoch):
        print('\nEpoch: %d' % epoch)
        net.train()
        train_loss = 0
        train_loss1 = 0
        train_loss2 = 0
        train_loss3 = 0
        train_loss4 = 0
        correct = 0
        total = 0
        idx = 0
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            idx = batch_idx
            if inputs.shape[0] < batch_size:
                continue
            if use_cuda:
                inputs, targets = inputs.to(device), targets.to(device)
            inputs, targets = Variable(inputs), Variable(targets)

            # update learning rate
            for nlr in range(len(optimizer.param_groups)):
                optimizer.param_groups[nlr]['lr'] = cosine_anneal_schedule(epoch, nb_epoch, lr[nlr])

            optimizer.zero_grad()
            inputs1 = patch_transformation(inputs, 4)
            output_1 = netp(inputs1)
            #print(output_1)
            #print(targets)
            loss1 = CELoss(output_1, targets) * 1
            loss1.backward()
            optimizer.step()


            _, predicted = torch.max(output_1.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

            train_loss +=  loss1.item()

            if batch_idx % 50 == 0:
                print(
                    'Step: %d | Loss1: %.3f | Acc: %.3f%% (%d/%d)' % (
                    batch_idx, train_loss / (batch_idx + 1), 
                    100. * float(correct) / total, correct, total))

        train_acc = 100. * float(correct) / total
        train_loss = train_loss / (idx + 1)
        with open(exp_dir + '/results_train.txt', 'a') as file:
            file.write(
                'Iteration %d | train_acc = %.5f | train_loss = %.5f |\n' % (
                epoch, train_acc, train_loss))

        if epoch < 40:
            val_acc, val_acc_com, val_loss = test(net, CELoss, 3)
            if val_acc_com > max_val_acc:
                max_val_acc = val_acc_com
                net.cpu()
                torch.save(net, './' + store_name + '/bestmodel.pth')
                net.to(device)
            net.cpu()
            torch.save(net, './' + store_name + '/'+str(epoch)+'model.pth')
            net.to(device)
            with open(exp_dir + '/results_test.txt', 'a') as file:
                file.write('Iteration %d, test_acc = %.5f, test_acc_combined = %.5f, test_loss = %.6f\n' % (
                epoch, val_acc, val_acc_com, val_loss))
        else:
            net.cpu()
            torch.save(net, './' + store_name + '/'+str(epoch)+'model.pth')
            net.to(device)

In [ ]:
train(nb_epoch=20,             # number of epoch
         batch_size=16,         # batch size
         store_name='',     # folder for output
         resume=False,          # resume training from checkpoint
         start_epoch=0,         # the start epoch number when you resume the training
         model_path='')

In [65]:
batch_size=8
transform_test = transforms.Compose([
        transforms.Scale((550, 550)),
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
testset=[]
testset = torchvision.datasets.ImageFolder(root='/home/data/newtrain/val',
                                               transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

In [66]:
len(testset)

4000

In [ ]:

PATH = "./style.pth" #7
net = torch.load(PATH)

net.to(torch.device("cuda:3"))

In [70]:
criterion = nn.CrossEntropyLoss()
test_loss = 0
correct = 0
correct_com = 0
total = 0
idx = 0

In [ ]:
    for batch_idx, (inputs, targets) in enumerate(testloader):
        idx = batch_idx
        if torch.cuda.is_available():
            inputs, targets = inputs.to(torch.device("cuda:3")), targets.to(torch.device("cuda:3"))
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        #print(targets)
        output_1= net(inputs)
        #print(output_1)

        loss = criterion(output_1, targets)

        test_loss += loss.item()
        _, predicted = torch.max(output_1.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        if batch_idx % 50 == 0:
            print('Step: %d | Loss: %.3f | Acc: %.3f%% (%d/%d)' % (
            batch_idx, test_loss / (batch_idx + 1), 100. * float(correct) / total, correct, total, ))

    test_acc = 100. * float(correct) / total
    test_acc_en = 100. * float(correct_com) / total
    test_loss = test_loss / (idx + 1)

In [15]:
test_acc#93.1625

98.92